# Metagenomics

## Intro

Vervuiling is een groot probleem in de moderne samenleving en een van de grootste bijdrager daarbij is plastic afval. Gelukkig is daar een alternatief voor gevonden: Bioplastic. Bioplastics zouden beter plastic zijn dan plastic, omdat in tegenstelling tot plastic bioplastics zijn gemaakt van biologisch afbreekbare materialen. PHA is hier een goed voorbeeld van, die breekt binnen 90 dagen helemaal af in basis grond. Maar dat betekend niet dat bioplastic perfect is, het kan nog steeds schadelijk zijn voor de grond waar het in beland. Daarom gaan wij hier onderzoeken wat voor effect PHA heeft op micro-organismen in de grond.


## Data

We hebben minION data met drie soorten sample: controle, met bioplastic pallets en met bioplastic poeder.

## Tool analyse

*8-3-2025 tot 11-3-2025*


### PEAR

PEAR kan worden geinstalleerd via bioconda. 
Afgeleidt van de [github pagina](https://github.com/tseemann/PEAR) en de [documentatie](https://cme.h-its.org/exelixis/web/software/pear/doc.html) lijkt het dat PEAR van illumina reads kijkt. 
In dit [artikel](https://cme.h-its.org/exelixis/web/software/pear/doc.html) staat dat PEAR kijkt naar overlap op foward en reverse reads en die samenvoegd, maar onze data bestaat alleen maar uit minION data. PEAR is dus niet relevant voor ons onderzoek

### Nanoplot
Nanoplot zit in bioconda.
Nanoplot neemt als input data afkomstig van nanopores(inculusief minION) in meerdere soorten format, bijv. fastq, sorted bam en fasta bestanden(https://github.com/wdecoster/NanoPlot). HIevan worden plots gemaakt en een html summary + een statestiek raport. 
Deze tool lijkt fastqc te vervangen voor nanopore data, omdat fastqc vrij langzaam kan zijn bij nanopore data.


### Optioneel: minIONQC
voor flow cell analyse, geeft extra betrouwbaarheid. Deze gebruikt een sequencing_summary.txt om figuren te genereren in te plaats van direct op de fastq files.

### pycoQC
gebruikt net als minIONQC sequencing_summary.txt om figuren te genereren. De figuren van pycoQC zijn interactief. Verder geeft het dezelfde soort plots als Nanoplot. [documentatie](https://a-slide.github.io/pycoQC/)

### welke van de drie?
Als basis is het goed om te kiezen tussen pycoQC en Nanoplot, minIONQC moet alleen gebruikt worden voor extra flow cell analyse. Het beste lijkt mij om gewoon Nanoplot te gebruiken, omdat er al wat flow cell analyse lijkt te zijn en dat niet heel relevant is voor ons project en nonoplot direct werkt op de fastq files, wat mij makkelijker lijkt.

*3-14-2025*

Begin inlezen minimap

als ref.mmi is er een bestand op assemblix onder "/data/dataprocessing/MinIONData/all_bacteria.mmi" die alle bacterie genomen bevat. 

voorbeeld commando:

```bash
minimap2 -a ref.mmi reads.fq > alignment.sam
```

https://github.com/lh3/minimap2
 

*3-16-2025*

Doel: maak minimap2 rule